In [42]:
import pyodbc
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL SERVER};"
    "SERVER=DESKTOP-RJ6N7AA\\MSSQLSERVER1;"
    "DATABASE=scdtype;"
    "UID=sa;"
    "PWD=1234567890;"
    "TrustServerCertificate=yes;"
)
from datetime import datetime


In [43]:
setup_sql = """
-- Clear old tables
IF OBJECT_ID('Customer_SCD0', 'U') IS NOT NULL DROP TABLE Customer_SCD0;
IF OBJECT_ID('Customer_SCD1', 'U') IS NOT NULL DROP TABLE Customer_SCD1;
IF OBJECT_ID('Customer_SCD2', 'U') IS NOT NULL DROP TABLE Customer_SCD2;
IF OBJECT_ID('Customer_SCD3', 'U') IS NOT NULL DROP TABLE Customer_SCD3;
IF OBJECT_ID('Customer_SCD4', 'U') IS NOT NULL DROP TABLE Customer_SCD4;
IF OBJECT_ID('Customer_SCD4_History', 'U') IS NOT NULL DROP TABLE Customer_SCD4_History;
IF OBJECT_ID('Customer_SCD5', 'U') IS NOT NULL DROP TABLE Customer_SCD5;
IF OBJECT_ID('Customer_SCD5_History', 'U') IS NOT NULL DROP TABLE Customer_SCD5_History;
IF OBJECT_ID('Customer_SCD6', 'U') IS NOT NULL DROP TABLE Customer_SCD6;

-- Type 0: Fixed data
CREATE TABLE Customer_SCD0 (
    CustomerID INT PRIMARY KEY,
    Name VARCHAR(100),
    Address VARCHAR(200)
);

-- Type 1: Replace data
CREATE TABLE Customer_SCD1 (
    CustomerID INT PRIMARY KEY,
    Name VARCHAR(100),
    Address VARCHAR(200)
);

-- Type 2: Save all changes
CREATE TABLE Customer_SCD2 (
    SurrogateKey INT IDENTITY(1,1) PRIMARY KEY,
    CustomerID INT,
    Name VARCHAR(100),
    Address VARCHAR(200),
    StartDate DATE,
    IsCurrent BIT
);

-- Type 3: Save last address
CREATE TABLE Customer_SCD3 (
    CustomerID INT PRIMARY KEY,
    Name VARCHAR(100),
    CurrentAddress VARCHAR(200),
    PreviousAddress VARCHAR(200)
);

-- Type 4: Current data
CREATE TABLE Customer_SCD4 (
    CustomerID INT PRIMARY KEY,
    Name VARCHAR(100),
    Address VARCHAR(200)
);

-- Type 4: Old data
CREATE TABLE Customer_SCD4_History (
    HistoryID INT IDENTITY(1,1) PRIMARY KEY,
    CustomerID INT,
    Name VARCHAR(100),
    Address VARCHAR(200),
    ChangeDate DATE
);

-- Type 5: Current with history link
CREATE TABLE Customer_SCD5 (
    CustomerID INT PRIMARY KEY,
    Name VARCHAR(100),
    Address VARCHAR(200),
    CurrentHistoryID INT
);

-- Type 5: Old data
CREATE TABLE Customer_SCD5_History (
    HistoryID INT IDENTITY(1,1) PRIMARY KEY,
    CustomerID INT,
    Name VARCHAR(100),
    Address VARCHAR(200),
    ChangeDate DATE
);

-- Type 6: Mix of 1, 2, 3
CREATE TABLE Customer_SCD6 (
    SurrogateKey INT IDENTITY(1,1) PRIMARY KEY,
    CustomerID INT,
    Name VARCHAR(100),
    CurrentAddress VARCHAR(200),
    PreviousAddress VARCHAR(200),
    StartDate DATE,
    IsCurrent BIT
);

-- Add one customer (John)
INSERT INTO Customer_SCD0 (CustomerID, Name, Address)
VALUES (1, 'John Doe', '123 Main St');

INSERT INTO Customer_SCD1 SELECT * FROM Customer_SCD0;
INSERT INTO Customer_SCD2 (CustomerID, Name, Address, StartDate, IsCurrent)
SELECT CustomerID, Name, Address, GETDATE(), 1 FROM Customer_SCD0;
INSERT INTO Customer_SCD3 SELECT CustomerID, Name, Address, NULL FROM Customer_SCD0;
INSERT INTO Customer_SCD4 SELECT * FROM Customer_SCD0;
INSERT INTO Customer_SCD5 (CustomerID, Name, Address) SELECT * FROM Customer_SCD0;
INSERT INTO Customer_SCD6 (CustomerID, Name, CurrentAddress, StartDate, IsCurrent)
SELECT CustomerID, Name, Address, GETDATE(), 1 FROM Customer_SCD0;
"""

In [44]:
cursor = conn.cursor()
for statement in setup_sql.split(';'):
    statement = statement.strip()
    if statement:
        cursor.execute(statement)
conn.commit()
print("Tables ready with two customers.")
cursor.execute("SELECT * FROM Customer_SCD1")
rows = cursor.fetchall()

for row in rows:
    print(row)


Tables ready with two customers.
(1, 'John Doe', '123 Main St')


In [45]:
customer_id=1
name='John Doe'
address='789 Pine Rd'


In [46]:
cursor.execute(
    "SELECT CustomerID FROM Customer_SCD1 WHERE CustomerID = ?",
    (customer_id,)                                                # pass params as a tuple
)
if cursor.fetchone():
    cursor.execute("UPDATE Customer_SCD1 SET Name = ?, Address = ? WHERE CustomerID = ?",
                   (name, address, customer_id))
else:
    cursor.execute("INSERT INTO Customer_SCD1 (CustomerID, Name, Address) VALUES (?, ?, ?)",
                   (customer_id, name, address))
print("Type 1: New address replaced old.")

Type 1: New address replaced old.


In [47]:
cursor.execute("SELECT * FROM Customer_SCD1")
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 'John Doe', '789 Pine Rd')


In [48]:
cursor.execute("SELECT * FROM Customer_SCD2")
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 1, 'John Doe', '123 Main St', datetime.date(2025, 6, 25), True)


In [49]:
# Type 2: Save new change
cursor.execute("SELECT Address FROM Customer_SCD2 WHERE CustomerID = ? AND IsCurrent = 1", customer_id)
current = cursor.fetchone()
if current and current[0] != address:
    cursor.execute("UPDATE Customer_SCD2 SET IsCurrent = 0 WHERE CustomerID = ? AND IsCurrent = 1", customer_id)
    cursor.execute("INSERT INTO Customer_SCD2 (CustomerID, Name, Address, StartDate, IsCurrent) VALUES (?, ?, ?, ?, 1)",
                   (customer_id, name, address, datetime.now().date()))
elif not current:
    cursor.execute("INSERT INTO Customer_SCD2 (CustomerID, Name, Address, StartDate, IsCurrent) VALUES (?, ?, ?, ?, 1)",
                   (customer_id, name, address, datetime.now().date()))
conn.commit()
print("Type 2: Saved new address.")

Type 2: Saved new address.


In [50]:
cursor.execute("SELECT * FROM Customer_SCD2")
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 1, 'John Doe', '123 Main St', datetime.date(2025, 6, 25), False)
(2, 1, 'John Doe', '789 Pine Rd', datetime.date(2025, 6, 25), True)


In [51]:
cursor.execute("SELECT * FROM Customer_SCD3")
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 'John Doe', '123 Main St', None)


In [52]:
# Type 3: Save last address
cursor.execute("SELECT CurrentAddress FROM Customer_SCD3 WHERE CustomerID = ?", customer_id)
current = cursor.fetchone()
if current and current[0] != address:
    cursor.execute("UPDATE Customer_SCD3 SET PreviousAddress = CurrentAddress, CurrentAddress = ? WHERE CustomerID = ?",
                   (address, customer_id))
elif not current:
    cursor.execute("INSERT INTO Customer_SCD3 (CustomerID, Name, CurrentAddress) VALUES (?, ?, ?)",
                   (customer_id, name, address))
conn.commit()
print("Type 3: Saved last address.")

Type 3: Saved last address.


In [53]:
cursor.execute("SELECT * FROM Customer_SCD3")
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 'John Doe', '789 Pine Rd', '123 Main St')


In [54]:
cursor.execute("SELECT * FROM Customer_SCD4")
rows = cursor.fetchall()

for row in rows:
    print(row)
    cursor.execute("SELECT * FROM Customer_SCD4_History")
rows = cursor.fetchall()

for row in rows:
    print(row)


(1, 'John Doe', '123 Main St')


In [55]:
cursor.execute("SELECT Address FROM Customer_SCD4 WHERE CustomerID = ?", customer_id)
current = cursor.fetchone()
if current and current[0] != address:
    cursor.execute("INSERT INTO Customer_SCD4_History (CustomerID, Name, Address, ChangeDate) VALUES (?, ?, ?, ?)",
                   (customer_id, name, current[0], datetime.now().date()))
    cursor.execute("UPDATE Customer_SCD4 SET Name = ?, Address = ? WHERE CustomerID = ?",
                   (name, address, customer_id))
elif not current:
    cursor.execute("INSERT INTO Customer_SCD4 (CustomerID, Name, Address) VALUES (?, ?, ?)",
                   (customer_id, name, address))
conn.commit()
print("Type 4: Updated current and history.")

Type 4: Updated current and history.


In [56]:
cursor.execute("SELECT * FROM Customer_SCD5")
rows = cursor.fetchall()

for row in rows:
    print(row)
    
cursor.execute("SELECT * FROM Customer_SCD5_History")
    
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 'John Doe', '123 Main St', None)


In [57]:
cursor.execute("SELECT Address FROM Customer_SCD5 WHERE CustomerID = ?", customer_id)
current = cursor.fetchone()
if current and current[0] != address:
    cursor.execute("INSERT INTO Customer_SCD5_History (CustomerID, Name, Address, ChangeDate) VALUES (?, ?, ?, ?)",
                   (customer_id, name, current[0], datetime.now().date()))
    cursor.execute("SELECT SCOPE_IDENTITY()")
    history_id = cursor.fetchone()[0]
    cursor.execute("UPDATE Customer_SCD5 SET Name = ?, Address = ?, CurrentHistoryID = ? WHERE CustomerID = ?",
                   (name, address, history_id, customer_id))
elif not current:
    cursor.execute("INSERT INTO Customer_SCD5 (CustomerID, Name, Address) VALUES (?, ?, ?)",
                   (customer_id, name, address))
conn.commit()
print("Type 5: Updated with history link.")

Type 5: Updated with history link.


In [58]:
cursor.execute("SELECT * FROM Customer_SCD5")
rows = cursor.fetchall()

for row in rows:
    print(row)
    
cursor.execute("SELECT * FROM Customer_SCD5_History")
    
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 'John Doe', '789 Pine Rd', 1)
(1, 1, 'John Doe', '123 Main St', datetime.date(2025, 6, 25))


In [59]:
# Type 6: Mix of 1, 2, 3
cursor.execute("SELECT CurrentAddress FROM Customer_SCD6 WHERE CustomerID = ? AND IsCurrent = 1", customer_id)
current = cursor.fetchone()
if current and current[0] != address:
    cursor.execute("UPDATE Customer_SCD6 SET IsCurrent = 0 WHERE CustomerID = ? AND IsCurrent = 1", customer_id)
    cursor.execute("INSERT INTO Customer_SCD6 (CustomerID, Name, CurrentAddress, PreviousAddress, StartDate, IsCurrent) VALUES (?, ?, ?, ?, ?, 1)",
                   (customer_id, name, address, current[0], datetime.now().date()))
elif not current:
    cursor.execute("INSERT INTO Customer_SCD6 (CustomerID, Name, CurrentAddress, StartDate, IsCurrent) VALUES (?, ?, ?, ?, 1)",
                   (customer_id, name, address, datetime.now().date()))
conn.commit()
print("Type 6: Updated with history and last address.")

# Close connection
cursor.close()
conn.close()

Type 6: Updated with history and last address.


In [60]:
cursor.execute("SELECT * FROM Customer_SCD6")
rows = cursor.fetchall()

for row in rows:
    print(row)


ProgrammingError: Attempt to use a closed cursor.